In [8]:
import sys
from pathlib import Path
import torch
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import importlib

# 讓 Python 找得到 src 資料夾
# 假設 notebook 在專案根目錄 (跟 src 同層)
sys.path.append(str(Path.cwd()))

# 設定繪圖風格
sns.set_theme(style="whitegrid")
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'Microsoft JhengHei']
plt.rcParams['axes.unicode_minus'] = False

In [9]:
# 取得目前 notebook 的路徑 (例如 D:\NCKU\paper4\script)
current_dir = Path.cwd()

# 取得上一層目錄 (專案根目錄，例如 D:\NCKU\paper4)
project_root = current_dir.parent

# 把根目錄加入 Python 的搜尋路徑
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
    print(f"已加入專案路徑: {project_root}")

# reload
to_delete = [m for m in sys.modules if m.startswith('src')]

for m in to_delete:
    del sys.modules[m]

print(f"已清理模組: {to_delete}")

import src.utils.data_loader as data_loader
import src.engine.trainer as trainer
import src.engine.evaluator as evaluator

# 硬體設定
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on: {DEVICE}")

已清理模組: ['src', 'src.utils', 'src.utils.data_loader', 'src.engine', 'src.models', 'src.models.layers', 'src.models.network', 'src.utils.metrics', 'src.engine.trainer', 'src.engine.evaluator']
Running on: cuda


In [ ]:
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# 全域設定
DATA_PATH = Path("../dataset/USD_TWD.csv")
HORIZON = 3
LOOKBACK = 30
EPOCHS = 100
LR = 0.001
SEED = 42
CNNEXPERT_KERNELSIZE = 5
SERIESDECOMPOSITION_KERNELSIZE = 15

# 設定隨機性 (Reproducibility)
torch.manual_seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)


In [11]:
if not DATA_PATH.exists():
    raise FileNotFoundError("找不到資料集！")

df = pd.read_csv(DATA_PATH)

# 載入資料
train_loader, test_loader, scalers, _, _, _, _ = data_loader.prepare_data(
    df, lookback=LOOKBACK, horizon=HORIZON
)

target_scaler = scalers['target']
print(f"Data ready. Train batches: {len(train_loader)}, Test batches: {len(test_loader)}")

Data ready. Train batches: 168, Test batches: 42


In [12]:
ablation_experiments = [
    {
        "name": "Full Model (Baseline)",
        "params": {}  # 空字典 = 全部使用預設值 (全開)
    },
    # {
    #     "name": "w/o CNN",
    #     "params": {"use_cnn": False}  # 關閉 CNN
    # },
    # {
    #     "name": "w/o Decomposition",
    #     "params": {"use_decomp": False}  # 關閉序列分解
    # },
    # 如果你有做 Gate 機制也可以測
    # {
    #     "name": "w/o Gating",
    #     "params": {"use_gate": False}
    # }
]

In [ ]:
results = []

print(f"開始執行 {len(ablation_experiments)} 組消融實驗...\n")

for i, exp in enumerate(ablation_experiments):
    exp_name = exp["name"]
    exp_params = exp["params"]

    print(f"[{i + 1}/{len(ablation_experiments)}] Running: {exp_name}")
    print(f"   Params: {exp_params}")

    # 1. 重置種子 (確保初始化權重一致)
    set_seed(42)

    # 2. 訓練模型
    # 注意：這裡使用了我們剛改好的 model_hyperparams 機制
    model = trainer.train_v11(
        train_loader=train_loader,
        test_loader=test_loader,
        device=DEVICE,
        horizon=HORIZON,
        num_epochs=EPOCHS,
        lr=LR,
        cnnExpert_KernelSize=CNNEXPERT_KERNELSIZE,
        seriesDecomposition_KernelSize=SERIESDECOMPOSITION_KERNELSIZE,
        model_hyperparams=exp_params
    )

    # 3. 評估模型
    metrics = evaluator.evaluate_model(
        model=model,
        test_loader=test_loader,
        scaler=target_scaler,
        device=DEVICE,
        horizon=HORIZON
    )

    # 4. 儲存結果
    # 假設 metrics 回傳 {'MSE': 0.1, 'MAE': 0.2, ...}
    record = {"Experiment": exp_name}
    record.update(metrics)
    results.append(record)

    print(f"   Result: MSE={metrics.get('MSE', 'N/A'):.4f}\n")

print("所有實驗結束！")

開始執行 1 組消融實驗...

[1/1] Running: Full Model (Baseline)
   Params: {}

[Training] Enhanced DLinear...
  Epoch 20 | Loss: 0.3197 | Trend W: -0.020 | Seas W: 0.079
  Epoch 40 | Loss: 0.2924 | Trend W: -0.039 | Seas W: 0.092
  Epoch 60 | Loss: 0.2749 | Trend W: -0.051 | Seas W: 0.109
  Epoch 80 | Loss: 0.2699 | Trend W: -0.060 | Seas W: 0.117


In [ ]:
# 轉成 DataFrame
df_res = pd.DataFrame(results)

# 顯示表格
print("\n=== Ablation Study Results ===")
display(df_res)  # Jupyter 裡用 display 比較漂亮

# 畫圖比較 (Bar Plot)
plt.figure(figsize=(10, 6))
# 這裡假設你的指標是 'MSE'，如果是其他請自行修改 y
sns.barplot(data=df_res, x='Experiment', y='MSE', hue='Experiment', palette='viridis', legend=False)

plt.title(f"Ablation Study (Horizon={HORIZON})", fontsize=15)
plt.ylabel("MSE (Lower is Better)")
plt.xlabel("Model Configuration")

# 在柱狀圖上標示數值
for index, row in df_res.iterrows():
    plt.text(index, row['MSE'], f"{row['MSE']:.4f}", color='black', ha="center", va="bottom")

plt.show()

# 存檔備份
df_res.to_csv("ablation_results.csv", index=False)